In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aynaaz/ayna_az.csv


In [2]:
import fasttext
import regex as re
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [3]:
df = pd.read_csv('/kaggle/input/aynaaz/ayna_az.csv')

In [4]:
df.head()

,title,categorie,date,text,url
0,\nBanklar üçün “əcəl zəngi”: sıradakı buyursun \n,İqtisadiyyat,\n27.10.2023 - 14:05\n,\nHüquqşünas: “Azərbaycanda banklara qarşı irə...,https://ayna.az/banklar-ucun-ecel-zengi-sirada...
1,\nÇörəyin ucuzlaşması söhbətləri və möhtəkirin...,İqtisadiyyat,\n30.10.2023 - 17:50\n,\nNiyə buğdanın qiymətinin aşağı düşməsi yalnı...,https://ayna.az/coreyin-ucuzlasmasi-sohbetleri...
2,\n“Azeri Light” ucuzlaşıb \n,İqtisadiyyat,\n24.10.2023 - 12:05\n,\nDünya birjalarında neft bahalaşıb\nDünya bir...,https://ayna.az/azeri-light-ucuzlasib-66390
3,\nNeft bahalaşıb \n,İqtisadiyyat,\n27.10.2023 - 11:42\n,\nAzərbaycan neftinin qiyməti 93 dollara yaxın...,https://ayna.az/neft-bahalasib-66502
4,\nNeft ucuzlaşıb \n,İqtisadiyyat,\n29.08.2023 - 11:24\n,\nAzərbaycan neftinin qiyməti dəyişməyib\nDüny...,https://ayna.az/neft-ucuzlasib-64310


In [5]:
df = df[['categorie','text']]

In [6]:
df.head()

,categorie,text
0,İqtisadiyyat,\nHüquqşünas: “Azərbaycanda banklara qarşı irə...
1,İqtisadiyyat,\nNiyə buğdanın qiymətinin aşağı düşməsi yalnı...
2,İqtisadiyyat,\nDünya birjalarında neft bahalaşıb\nDünya bir...
3,İqtisadiyyat,\nAzərbaycan neftinin qiyməti 93 dollara yaxın...
4,İqtisadiyyat,\nAzərbaycan neftinin qiyməti dəyişməyib\nDüny...


In [7]:
df['text'] = df['text'].str.replace('\n',' ')

In [8]:
df['categorie'].value_counts()

categorie
Xəbərlər            21533
Dünya               13327
Cəmiyyət             6780
Siyasət              5515
Hadisə               3597
Aktual               2874
İqtisadiyyat         2112
İdman                1567
Mədəniyyət            751
Maraqlı               711
Tarix                  75
Reklam                 35
İbrahim Nəbioğlu       30
Arif Əliyev            14
Elçin Şıxlı            14
Yazarlar               10
Qulu Məhərrəmli         7
Name: count, dtype: int64

In [9]:
df['categorie'] = '__label__'+df['categorie']

In [10]:
train, test = train_test_split(df,test_size= 0.1)

In [11]:
train

,categorie,text
51654,__label__Dünya,Ermənistanın müdafiə naziri istefa verib Er...
53441,__label__Dünya,"Sürüşmə nəticəsində ölənlərlə yanaşı, 4 nəfər..."
41469,__label__Aktual,"""Pilotların eniş zamanı yol verdikləri kobud ..."
14003,__label__Xəbərlər,Qəbul prosesi elektron qaydada həyata keçiril...
51473,__label__Dünya,Koronavirus can almaqda davam edir Türkiyə...
...,...,...
892,__label__İqtisadiyyat,Nazir: “Ölkə Prezidentinin tapşırığı ilə maaş...
29332,__label__Siyasət,Azad olunmuş daha bir kəndimizdən görüntülər ...
18489,__label__Xəbərlər,Əvvəlki dövrlərdən bağlı qalmış səkkiz cinayə...
16838,__label__Xəbərlər,Nazirlik mövzuları “Dərs vaxtı”nda öyrənməyi ...


In [12]:
train.to_csv(r'train.txt', header = None,index =None,sep=' ')
test.to_csv(r'test.txt', header = None,index =None,sep=' ')

In [13]:
model = fasttext.train_supervised('train.txt')

Read 11M words
Number of words:  493497
Number of labels: 13
Progress: 100.0% words/sec/thread: 1598794 lr:  0.000000 avg.loss:  0.864743 ETA:   0h 0m 0s


In [14]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('test.txt'))

N	5886
P@1	0.761
R@1	0.761


In [15]:
model.predict('Çoxdan gözlənilən Qızıl top mükafatı öz qalibini tapdı. Argentinalı ulduz Lionel Messi karyerasında 8-ci dəfə bu nəhəng mükafatı qazanıb. Ancaq Messinin bu mükafatı qazanmasınə bəzi şəxslər də tənqid edib. Kriştiano Ronaldo səkkizinci dəfə "Qızıl top"u qazanan Lionel Messi ilə bağlı tənqidlərlə dolu paylaşıma münasibət bildirib.')

(('__label__İqtisadiyyat',), array([0.26582932]))

In [16]:
tst = [model.predict(test['text'].iloc[i])[0][0] for i in range(len(test['text']))]

In [17]:
print(accuracy_score(tst, test['categorie']))

0.7564450474898237
